#  PikaPikaGenerator - Data Preprocessing
# 
**Progetto:** Generative Synthesis of Pokémon Sprites from Textual Descriptions  
 **Corso:** Deep Learning - Politecnico di Bari  
 **Studente:** Pasquale Alessandro Denora  
 **Professore:** Vito Walter Anelli 

#   Import e Setup Iniziale
 
 Il file inizia importando tutte le librerie necessarie per il preprocessing dei dati Pokemon. Le librerie principali includono:
 - **pandas**: per manipolare il CSV con i dati Pokemon
 - **PIL**: per caricare e validare le immagini sprite  
 - **torch**: per creare Dataset e DataLoader PyTorch
 - **transformers**: per il tokenizer BERT
 - **pathlib**: per gestire i percorsi dei file


In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from typing import Dict, List, Tuple, Optional
import logging
from pathlib import Path
import json
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

#  Classe PokemonDataProcessor - Inizializzazione
 
 La classe principale `PokemonDataProcessor` gestisce tutto il processo di preprocessing. Nel costruttore (righe 21-25) vengono inizializzati:
 - `self.config`: la configurazione del progetto
 - `self.df`: il DataFrame pandas (inizialmente None)
 - `self.tokenizer`: il tokenizer BERT (inizialmente None)  
 - `self.stats`: dizionario per raccogliere statistiche


In [ ]:
class PokemonDataProcessor:
    
    def __init__(self, config: Dict):
        self.config = config
        self.df = None
        self.tokenizer = None
        self.stats = {}


#   Caricamento e Validazione CSV

 Il metodo `load_and_validate_data()` (righe 27-66) carica il file CSV Pokemon. La strategia è robusta:
 
 1. **Prova multiple encoding** (righe 32-46): Il file potrebbe avere encoding diversi, quindi prova in sequenza utf-16, utf-8, cp1252, latin1
 2. **Separatore tab** (riga 38): Usa `sep='\t'` perché il file è tab-separated
 3. **Validazione colonne** (righe 49-52): Verifica che esistano le colonne richieste
 4. **Pulizia dati** (righe 55-60): Rimuove righe senza descrizione o con descrizioni troppo corte (<10 caratteri)

In [ ]:
def load_and_validate_data(self) -> pd.DataFrame:
    """Load and validate Pokemon CSV data"""
    logger.info("Loading Pokemon dataset...")
    
    # prova più encodings 
    encodings = ['utf-16', 'utf-8', 'cp1252', 'latin1']
    for encoding in encodings:
        try:
            self.df = pd.read_csv(
                self.config['data']['raw_data_path'], 
                encoding=encoding,
                sep='\t'
            )
            logger.info(f"Successfully loaded with encoding: {encoding}")
            break
        except Exception as e:
            continue
    
    if self.df is None:
        raise ValueError("Could not load CSV with any encoding")
    
    # Convalida le columns
    required_cols = ['description', 'english_name', 'national_number']
    missing_cols = [col for col in required_cols if col not in self.df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Pulisce i data
    initial_count = len(self.df)
    self.df = self.df.dropna(subset=['description'])
    self.df = self.df[self.df['description'].str.len() > 10]  # Rimuovi descrizioni troppo corte
    final_count = len(self.df)
    
    logger.info(f"Loaded {final_count} valid Pokemon (removed {initial_count - final_count})")
    
    # Store statistics
    self.stats['total_pokemon'] = final_count
    self.stats['description_lengths'] = self.df['description'].str.len().describe().to_dict()
    
    return self.df

#   Validazione Immagini Pokemon

 Il metodo `validate_images()` (righe 68-109) è cruciale perché verifica che ogni Pokemon nel CSV abbia un'immagine corrispondente:
 
 1. **Percorsi multipli** (righe 80-84): Per ogni Pokemon prova diversi formati di nome file:
    - `001.png` (numero con zero padding) 
    - `small_images/001.png` (sottodirectory)
    - `1.png` (numero senza padding)
 
 2. **Validazione integrità** (righe 91-95): Non basta che il file esista, deve essere un'immagine valida
    - Usa `Image.open()` e `img.verify()` per controllare l'integrità
 
 3. **Tracciamento risultati**: Mantiene un dizionario delle immagini valide e una lista di quelle mancanti


In [ ]:
def validate_images(self) -> Dict[int, str]:
    """Validate available images and create mapping"""
    logger.info("Validating Pokemon images...")
    
    images_dir = Path(self.config['data']['images_dir'])
    valid_images = {}
    missing_images = []
    
    for idx, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Validating images"):
        national_number = int(row['national_number'])
        
        # Controlla possibili percorsi dell'immagine
        possible_paths = [
            images_dir / f"{national_number:03d}.png",
            images_dir / "small_images" / f"{national_number:03d}.png",
            images_dir / f"{national_number}.png"
        ]
        
        image_found = False
        for img_path in possible_paths:
            if img_path.exists():
                try:
                    # Valida immagini che siano immagini valide
                    img = Image.open(img_path)
                    img.verify()
                    valid_images[idx] = str(img_path)
                    image_found = True
                    break
                except:
                    continue
        
        if not image_found:
            missing_images.append(national_number)
    
    logger.info(f"Found {len(valid_images)} valid images")
    if missing_images:
        logger.warning(f"Missing images for {len(missing_images)} Pokemon: {missing_images[:10]}...")
    
    self.stats['valid_images'] = len(valid_images)
    self.stats['missing_images'] = len(missing_images)
    
    return valid_images

#   Creazione Split Train/Validation/Test

 Il metodo `create_splits()` (righe 111-138) divide il dataset in tre parti per il machine learning:
 
 1. **Filtraggio** (riga 116): Lavora solo sui Pokemon che hanno immagini valide
 2. **Shuffling riproducibile** (righe 119-121): 
    - Imposta il seed da config per riproducibilità
    - Mischia gli indici in modo random ma deterministic
 3. **Calcolo proporzioni** (righe 124-126): Calcola le dimensioni basandosi sui parametri in config:
    - `config['data']['train_split']` (es. 0.8 = 80%)
    - `config['data']['val_split']` (es. 0.1 = 10%)
    - Il resto va automaticamente a test
 4. **Slicing sequenziale**: Divide la lista shuffled in tre parti consecutive


In [ ]:
def create_splits(self, valid_indices: List[int]) -> Dict[str, List[int]]:
    """Create train/val/test splits"""
    logger.info("Creating data splits...")
    
    # Filter to only valid entries
    valid_df = self.df.loc[valid_indices]
    
    # Shuffle
    indices = valid_df.index.tolist()
    np.random.seed(self.config['project']['seed'])
    np.random.shuffle(indices)
    
    # Calculate split sizes
    n_total = len(indices)
    n_train = int(n_total * self.config['data']['train_split'])
    n_val = int(n_total * self.config['data']['val_split'])
    
    # Create splits
    splits = {
        'train': indices[:n_train],
        'val': indices[n_train:n_train + n_val],
        'test': indices[n_train + n_val:]
    }
    
    logger.info(f"Split sizes - Train: {len(splits['train'])}, "
               f"Val: {len(splits['val'])}, Test: {len(splits['test'])}")
    
    return splits


#   Salvataggio Dati Processati

 Il metodo `save_processed_data()` (righe 140-160) salva tutto il lavoro fatto in file JSON e CSV:
 
 1. **Creazione directory** (righe 143): Crea la directory di output se non esiste
 2. **File salvati**:
    - `valid_images.json`: mappa indice → percorso immagine
    - `splits.json`: indici per train/val/test  
    - `processed_pokemon.csv`: DataFrame pulito
    - `dataset_stats.json`: statistiche complete
 
 Tutti i percorsi usano `config['data']['processed_data_path']` come directory base.


In [ ]:
def save_processed_data(self, valid_images: Dict, splits: Dict):
    """Save processed data and metadata"""
    output_dir = Path(self.config['data']['processed_data_path'])
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Salva mappatura delle immagini valide
    with open(output_dir / 'valid_images.json', 'w') as f:
        json.dump(valid_images, f)
    
    # Salva gli indici delle suddivisioni
    with open(output_dir / 'splits.json', 'w') as f:
        json.dump(splits, f)
    
    # Salva il dataframe processato
    self.df.to_csv(output_dir / 'processed_pokemon.csv', index=True)
    
    # Salva le statistiche del dataset
    with open(output_dir / 'dataset_stats.json', 'w') as f:
        json.dump(self.stats, f, indent=2)
    
    logger.info(f"Saved processed data to {output_dir}")


#  Pipeline Completa di Preprocessing
 
 Il metodo `process()` (righe 162-180) orchestra l'intera pipeline:
 
 1. Carica e valida il CSV
 2. Valida tutte le immagini  
 3. Crea gli split train/val/test
 4. Salva tutto nei file di output
 
 È il metodo principale che viene chiamato da `main.py` quando si esegue il preprocessing.


In [ ]:
def process(self):
    """Run complete preprocessing pipeline"""
    logger.info("Starting data preprocessing...")
    
    # Carica e valida i dati
    self.load_and_validate_data()
    
    # Convalida le immagini
    valid_images = self.validate_images()
    valid_indices = list(valid_images.keys())
    
    # Crea le suddivisioni
    splits = self.create_splits(valid_indices)
    
    # Salva i dati processati
    self.save_processed_data(valid_images, splits)
    
    logger.info("Data preprocessing completed!")
    return self.stats

#   Classe PokemonDataset per PyTorch

 La classe `PokemonDataset` (righe 183-223) eredita da `torch.utils.data.Dataset` e prepara i dati per il training:
 
 **Inizializzazione (righe 200-226)**:
 - Carica i dati processati dai file JSON/CSV salvati prima
 - Inizializza il tokenizer BERT specificato in config
 - Configura le trasformazioni per le immagini
 
 **Parametri importanti**:
 - `image_size`: dimensione target per le immagini (da config)
 - `max_length`: lunghezza massima sequenze per il tokenizer
 - `augment`: se applicare data augmentation (solo per training)


In [ ]:
class PokemonDataset(Dataset):
    """PyTorch Dataset for Pokemon sprite generation"""
    
    def __init__(
        self, 
        data_path: str,
        split: str,
        tokenizer_name: str,
        max_length: int = 128,
        image_size: int = 215,
        augment: bool = False
    ):
        super().__init__()
        self.split = split
        self.max_length = max_length
        self.image_size = image_size
        self.augment = augment
        
        # Carica i dati processati
        processed_dir = Path(data_path)
        
        # Carica il dataframe 
        self.df = pd.read_csv(processed_dir / 'processed_pokemon.csv', index_col=0)
        
        # Carica la mappatura delle immagini valide
        with open(processed_dir / 'valid_images.json', 'r') as f:
            self.valid_images = {int(k): v for k, v in json.load(f).items()}
        
        # Carica le suddivisioni
        with open(processed_dir / 'splits.json', 'r') as f:
            splits = json.load(f)
            self.indices = splits[split]
        
        # Inizializza il tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        
        # Setta le trasformazioni delle immagini
        self._setup_transforms()
        
        logger.info(f"Loaded {split} dataset with {len(self.indices)} samples")


#   Setup Trasformazioni Immagini

 Il metodo `_setup_transforms()` (righe 224-246) configura come preprocessare le immagini:
 
 **Trasformazioni base** (sempre applicate):
 - `Resize`: ridimensiona a dimensione target
 - `ToTensor`: converte PIL Image in tensor PyTorch  
 - `Normalize`: normalizza pixel da [0,1] a [-1,1] usando mean=0.5, std=0.5
 
 **Data Augmentation** (solo per training):
 - `RandomCrop`: crop casuale dopo resize più grande
 - `RandomHorizontalFlip`: flip orizzontale casuale
 - `ColorJitter`: variazioni casuali di luminosità, contrasto, saturazione


In [ ]:
def _setup_transforms(self):
    """Setup image transformations"""
    import torchvision.transforms as transforms
    
    # Definisci le trasformazioni di base
    basic_transforms = [
        transforms.Resize((self.image_size, self.image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]
    
    if self.augment and self.split == 'train':
        # Aggiungi trasformazioni di data augmentation
        self.transform = transforms.Compose([
            transforms.Resize((self.image_size + 20, self.image_size + 20)),
            transforms.RandomCrop(self.image_size),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
    else:
        self.transform = transforms.Compose(basic_transforms)

#   Metodo __getitem__ - Caricamento Singolo Sample

 Il metodo `__getitem__()` (righe 251-285) è chiamato dal DataLoader per ottenere un singolo sample:
 
 **Processo step-by-step**:
 1. **Recupera dati Pokemon** (righe 253-254): Usa l'indice per ottenere la riga corrispondente dal DataFrame
 2. **Carica immagine** (righe 257-258): Apre l'immagine dal percorso salvato
 3. **Gestione trasparenza** (righe 260-264): Converte RGBA→RGB usando background bianco
 4. **Trasformazioni** (riga 266): Applica resize, normalizzazione, augmentation
 5. **Tokenizzazione testo** (righe 269-276): Tokenizza la descrizione Pokemon con padding/truncation
 
 **Output**: Dizionario con image tensor, token IDs, attention mask, testo originale, nome Pokemon


In [ ]:
def __getitem__(self, idx):
    # Ottiene l'indice del dataframe
    df_idx = self.indices[idx]
    row = self.df.loc[df_idx]
    
    # Carica e processa l'immagine
    image_path = self.valid_images[df_idx]
    image = Image.open(image_path).convert('RGBA')
    
    # Converti RGBA a RGB se necessario
    if image.mode == 'RGBA':
        background = Image.new('RGB', image.size, (255, 255, 255))
        background.paste(image, mask=image.split()[3])
        image = background
    
    image = self.transform(image)
    
    # Tokenizza il testo
    text = row['description']
    encoding = self.tokenizer(
        text,
        max_length=self.max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    return {
        'image': image,
        'input_ids': encoding['input_ids'].squeeze(0),
        'attention_mask': encoding['attention_mask'].squeeze(0),
        'text': text,
        'name': row['english_name'],
        'idx': df_idx
    }

#   Funzione create_dataloaders
 
 La funzione `create_dataloaders()` (righe 290-316) crea i DataLoader PyTorch per train/val/test:
 
 **Configurazione per ogni split**:
 - Legge `image_size` da config con fallback a 215
 - Crea PokemonDataset per ogni split (train/val/test)
 - Applica augmentation solo al training set
 
 **Parametri DataLoader** (da config):
 - `batch_size`: numero di sample per batch
 - `num_workers`: processi per caricamento parallelo
 - `shuffle=True` solo per training
 - `pin_memory=True` se GPU disponibile (ottimizzazione)
 - `drop_last=True` solo per training (batch uniformi)


In [ ]:
def create_dataloaders(config: Dict, tokenizer_name: str) -> Dict[str, DataLoader]:
    """Create dataloaders for all splits"""
    dataloaders = {}
    
    # Leggi image_size dal config con fallback
    image_size = config.get('data', {}).get('image_size', 215)
    
    for split in ['train', 'val', 'test']:
        dataset = PokemonDataset(
            data_path=config['data']['processed_data_path'],
            split=split,
            tokenizer_name=tokenizer_name,
            max_length=config['model']['encoder']['max_length'],
            image_size=image_size,
            augment=True if split == 'train' else False
        )
        
        dataloaders[split] = DataLoader(
            dataset,
            batch_size=config['training']['batch_size'],
            shuffle=True if split == 'train' else False,
            num_workers=config['training']['num_workers'],
            pin_memory=torch.cuda.is_available(),
            drop_last=True if split == 'train' else False
        )
    
    return dataloaders

#   Script di Test
 
 Il blocco finale (righe 321-330) permette di testare il preprocessing quando si esegue il file direttamente:
 
 1. Carica la configurazione da `configs/config.yaml`
 2. Crea un'istanza di PokemonDataProcessor  
 3. Esegue la pipeline completa con `processor.process()`
 4. Stampa le statistiche finali
 
 Questo è utile per debug e per verificare che tutto funzioni prima di integrare nel training.


In [ ]:
if __name__ == "__main__":
    # Test preprocessing
    import yaml
    
    with open('configs/config.yaml', 'r') as f:
        config = yaml.safe_load(f)
    
    processor = PokemonDataProcessor(config)
    stats = processor.process()
    
    print("\nDataset Statistics:")
    print(json.dumps(stats, indent=2))

#  Riepilogo Parametri Config Utilizzati
 
 Il file `preprocessing.py` fa riferimento a questi parametri del file `config.yaml`:
 
 **Sezione `data`**:
 - `raw_data_path`: percorso del file CSV Pokemon
 - `images_dir`: directory contenente le sprite Pokemon  
 - `processed_data_path`: directory di output per dati processati
 - `image_size`: dimensione target per le immagini (320px nel tuo config)
 - `train_split`, `val_split`, `test_split`: proporzioni degli split
 
 **Sezione `model.encoder`**:
 - `model_name`: nome del tokenizer BERT ("prajjwal1/bert-mini")
 - `max_length`: lunghezza massima sequenze token (128)
 
 **Sezione `training`**:
 - `batch_size`: dimensione batch per DataLoader (2)
 - `num_workers`: processi paralleli per caricamento dati (1)
 
 **Sezione `project`**:
 - `seed`: seed per riproducibilità (42)
 
 Il preprocessing è completamente configurabile attraverso questi parametri, rendendo facile sperimentare con diverse impostazioni.


#  Conclusioni
 
 Il file `preprocessing.py` implementa una pipeline robusta e completa per preparare i dati Pokemon:
 
 1. **Caricamento intelligente**: Gestisce diversi encoding CSV
 2. **Validazione rigorosa**: Controlla sia dati che immagini
 3. **Split riproducibili**: Usa seed fisso per risultati consistenti  
 4. **Dataset PyTorch pronto**: Integrazione diretta con training loop
 5. **Configurazione flessibile**: Tutti i parametri leggibili da config.yaml
 
 La struttura modulare permette di:
 - Eseguire solo parti specifiche della pipeline
 - Facilmente debug e modificare comportamenti
 - Riutilizzare componenti in altri progetti
 
 Il risultato è un dataset pulito e standardizzato, pronto per alimentare il modello encoder-decoder del progetto PikaPikaGenerator.
